In [1]:
import tensorflow as tf
import string
import requests
import pandas as pd

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout

## Load Dataset

In [2]:
df = pd.read_csv('Datasets/Spanish-Story-Dataset.txt', delimiter = "\n", header = None)

In [3]:
Text = df[0].tolist()

In [4]:
len(Text)

2040

In [5]:
len(" ".join(Text))

118900

## Build LSTM Model and Prepare X and y

In [6]:
token = Tokenizer()
token.fit_on_texts(Text)

In [7]:
token.word_index

{'el': 1,
 'y': 2,
 'a': 3,
 'que': 4,
 'se': 5,
 'la': 6,
 'de': 7,
 'no': 8,
 'le': 9,
 'su': 10,
 'lo': 11,
 'en': 12,
 'un': 13,
 'los': 14,
 'al': 15,
 'me': 16,
 'ya': 17,
 'dice': 18,
 'una': 19,
 'por': 20,
 'con': 21,
 'te': 22,
 'para': 23,
 'coyote': 24,
 'cuando': 25,
 'como': 26,
 'señor': 27,
 'hombre': 28,
 'muy': 29,
 'así': 30,
 'voy': 31,
 'casa': 32,
 'pero': 33,
 'qué': 34,
 'está': 35,
 'les': 36,
 'yo': 37,
 'es': 38,
 'dijo': 39,
 'las': 40,
 'estaba': 41,
 'aquí': 42,
 'donde': 43,
 'bien': 44,
 'si': 45,
 'ahí': 46,
 'ahora': 47,
 'dinero': 48,
 'sí': 49,
 'fue': 50,
 'va': 51,
 'había': 52,
 'del': 53,
 'comer': 54,
 'muchacho': 55,
 'más': 56,
 'tlacuache': 57,
 'todo': 58,
 'día': 59,
 'sólo': 60,
 'tenía': 61,
 'sus': 62,
 'esposa': 63,
 'vez': 64,
 'mucho': 65,
 'este': 66,
 'ver': 67,
 'mi': 68,
 '¿qué': 69,
 'él': 70,
 'boa': 71,
 'todos': 72,
 'hijo': 73,
 'porque': 74,
 'van': 75,
 'vamos': 76,
 'eso': 77,
 'hasta': 78,
 'nada': 79,
 'animales': 80,
 '

In [8]:
encoded_text = token.texts_to_sequences(Text)

In [9]:
encoded_text

[[759, 336, 760, 4, 12, 1, 211, 761, 1501],
 [1502, 337, 1503, 2, 4, 338, 7, 762, 212, 388, 1504],
 [4, 1505, 1, 999, 763, 7, 13, 28],
 [87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20, 619, 5, 445],
 [1507, 58, 1, 178, 198, 389, 2, 25, 620, 213],
 [1508, 20, 6, 514, 19, 1000, 7, 1001, 4, 5, 1002, 20],
 [390, 1509, 20, 66, 263, 1510, 81, 1511, 3, 14, 446, 1003, 187],
 [1512, 4, 5, 515, 21, 92, 1513, 2, 5, 14, 1514],
 [93, 621, 16, 6, 622, 68, 1515, 2, 1516, 96, 764, 30, 1517],
 [18, 4, 94, 391, 171, 13, 1518, 7, 1519, 3, 96, 2, 3, 10, 73],
 [36, 392, 1, 199, 339],
 [92, 59, 227, 73, 5, 623, 291, 37, 41, 1520, 5, 16, 292, 2, 16, 39],
 [1004, 4, 17, 264, 315, 7, 765, 624, 4, 1005, 66, 393, 4],
 [1006],
 [37, 9, 1007],
 [1008, 516, 1521, 766, 316, 624, 69, 50, 11, 4, 516],
 [70, 16, 265],
 [1522, 4, 1523, 3, 214, 15, 211, 2, 12, 1, 113, 16, 1009, 21],
 [19, 120, 1010, 7, 1524, 13, 1525, 53, 340, 394, 4, 1011],
 [10, 200, 83, 29, 1012, 61, 1, 767, 1013, 1526, 2],
 [16, 239, 4, 3, 240, 88, 9, 15

In [10]:
vocab_size = len(token.word_counts) + 1

In [11]:
print(vocab_size)

3372


## Prepare Training Data

In [12]:
datalist = []
for d in encoded_text:
    if len(d) > 1:#skip 1 len sentence 
        for i in range(2, len(d)):
            datalist.append(d[:i])
            print(d[:i])

[759, 336]
[759, 336, 760]
[759, 336, 760, 4]
[759, 336, 760, 4, 12]
[759, 336, 760, 4, 12, 1]
[759, 336, 760, 4, 12, 1, 211]
[759, 336, 760, 4, 12, 1, 211, 761]
[1502, 337]
[1502, 337, 1503]
[1502, 337, 1503, 2]
[1502, 337, 1503, 2, 4]
[1502, 337, 1503, 2, 4, 338]
[1502, 337, 1503, 2, 4, 338, 7]
[1502, 337, 1503, 2, 4, 338, 7, 762]
[1502, 337, 1503, 2, 4, 338, 7, 762, 212]
[1502, 337, 1503, 2, 4, 338, 7, 762, 212, 388]
[4, 1505]
[4, 1505, 1]
[4, 1505, 1, 999]
[4, 1505, 1, 999, 763]
[4, 1505, 1, 999, 763, 7]
[4, 1505, 1, 999, 763, 7, 13]
[87, 5]
[87, 5, 18]
[87, 5, 18, 4]
[87, 5, 18, 4, 338]
[87, 5, 18, 4, 338, 212]
[87, 5, 18, 4, 338, 212, 29]
[87, 5, 18, 4, 338, 212, 29, 1506]
[87, 5, 18, 4, 338, 212, 29, 1506, 2]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20, 619]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20, 619, 5]
[1507, 58]
[1507, 58, 1]
[1507, 58, 1, 178]
[1507, 58, 1, 178, 198]
[1507, 58, 1, 178

## Padding

In [13]:
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
sequences

array([[  0,   0,   0, ...,   0, 759, 336],
       [  0,   0,   0, ..., 759, 336, 760],
       [  0,   0,   0, ..., 336, 760,   4],
       ...,
       [  0,   0,   0, ...,  76, 432,   3],
       [  0,   0,   0, ...,   0,  23,   4],
       [  0,   0,   0, ...,  23,   4,  86]], dtype=int32)

In [14]:
X = sequences[:, :-1]

In [15]:
y = sequences[:, -1]

In [16]:
y = to_categorical(y, num_classes=vocab_size)

In [17]:
y #one hot encoder

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
seq_length = X.shape[1]

In [19]:
seq_length

19

## LSTM Model Training

In [56]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            168600    
_________________________________________________________________
lstm (LSTM)                  (None, 19, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 3372)              340572    
Total params: 660,072
Trainable params: 660,072
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
model.fit(X, y, batch_size=32, epochs=100)


Epoch 1/100
567/567 [==============================] - 25s 34ms/step - loss: 6.7207 - accuracy: 0.0420
Epoch 2/100
567/567 [==============================] - 19s 33ms/step - loss: 5.9980 - accuracy: 0.0427
Epoch 3/100
567/567 [==============================] - 19s 33ms/step - loss: 5.7644 - accuracy: 0.0573
Epoch 4/100
567/567 [==============================] - 18s 33ms/step - loss: 5.6124 - accuracy: 0.0670
Epoch 5/100
567/567 [==============================] - 19s 33ms/step - loss: 5.5025 - accuracy: 0.0775
Epoch 6/100
567/567 [==============================] - 19s 33ms/step - loss: 5.3916 - accuracy: 0.0836
Epoch 7/100
567/567 [==============================] - 20s 35ms/step - loss: 5.2590 - accuracy: 0.0925
Epoch 8/100
567/567 [==============================] - 20s 35ms/step - loss: 5.2019 - accuracy: 0.0961
Epoch 9/100
567/567 [==============================] - 19s 33ms/step - loss: 5.1015 - accuracy: 0.1050
Epoch 10/100
567/567 [==============================] - 19s 33ms/step - l

In [60]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [61]:
seed_text = 'Había una '
generate_poetry(seed_text, 50)

vez un ratón que otros insectos por regresar a ver el muchacho le dice a su esposa y va cantar
y no tienen nada en pleno se quedó dormido sobre el camino y grande fuerte y se puso a reír
el señor dice mesa pon el mantel y sin elotes en el cerro y se va la señora se sentó
menos me voy a morder te voy a comer y le dice el hombre dice ¡pobre coyote empezaron y mañana
lo que le gustaba andar engañando a sus hermosas grandes pasaba naranjas que él se asustó y luego se puso
a andar mañana a la mañana siguiente muy lejos la boa le dice ahora sí te voy a mucha engañando
el hombre dice ahorita lo que se va a pasar a los demás porque ya no te di y el
hombre le dice a su esposa y se va a pasar la señora lleva todo lo que se trataba y
le dice el hombre dice ahorita lo que se va a pasar la señora lleva todo lo que se vendía
se adelantó y lo levantó como adultos ruidos le dice el hombre dice ahorita lo que digo y y de
inmediato se va a traer la circunstancia del señor y lo hace el tlacuache le amarr

In [20]:
model2 = Sequential()
model2.add(Embedding(vocab_size, 50, input_length=seq_length))
model2.add(LSTM(256, return_sequences=True))
model2.add(Dropout(0.1))
model2.add(LSTM(512,return_sequences=True))
model2.add(Dropout(0.1))
model2.add(LSTM(1024,return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(512))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(vocab_size, activation='softmax'))

In [21]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            168600    
_________________________________________________________________
lstm (LSTM)                  (None, 19, 256)           314368    
_________________________________________________________________
dropout (Dropout)            (None, 19, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 19, 512)           1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 19, 1024)          6295552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 19, 1024)          0

In [23]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model2.fit(X, y, batch_size=32, epochs=100)


Epoch 1/100
567/567 [==============================] - 37s 48ms/step - loss: 6.6767 - accuracy: 0.0340
Epoch 2/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1890 - accuracy: 0.0368
Epoch 3/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1737 - accuracy: 0.0374
Epoch 4/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1284 - accuracy: 0.0355
Epoch 5/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1602 - accuracy: 0.0380
Epoch 6/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1554 - accuracy: 0.0417
Epoch 7/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1510 - accuracy: 0.0411
Epoch 8/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1569 - accuracy: 0.0394
Epoch 9/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1594 - accuracy: 0.0389
Epoch 10/100
567/567 [==============================] - 28s 49ms/step - l

In [27]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model2.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [28]:
seed_text = 'Había una '
generate_poetry(seed_text, 50)

y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y


In [29]:
model3 = Sequential()
model3.add(Embedding(vocab_size, 50, input_length=seq_length))
model3.add(LSTM(256, return_sequences=True))
model3.add(Dropout(0.2))
model3.add(LSTM(256))
model3.add(Dropout(0.2))
model3.add(Dense(vocab_size, activation='softmax'))

In [30]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 50)            168600    
_________________________________________________________________
lstm_4 (LSTM)                (None, 19, 256)           314368    
_________________________________________________________________
dropout_3 (Dropout)          (None, 19, 256)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3372)              866604    
Total params: 1,874,884
Trainable params: 1,874,884
Non-trainable params: 0
____________________________________________

In [31]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model3.fit(X, y, batch_size=32, epochs=150)


Epoch 1/150
567/567 [==============================] - 10s 12ms/step - loss: 6.6055 - accuracy: 0.0373
Epoch 2/150
567/567 [==============================] - 7s 12ms/step - loss: 6.0861 - accuracy: 0.0456
Epoch 3/150
567/567 [==============================] - 7s 12ms/step - loss: 5.8336 - accuracy: 0.0684
Epoch 4/150
567/567 [==============================] - 7s 12ms/step - loss: 5.6761 - accuracy: 0.0747
Epoch 5/150
567/567 [==============================] - 7s 13ms/step - loss: 5.5214 - accuracy: 0.0869
Epoch 6/150
567/567 [==============================] - 7s 12ms/step - loss: 5.4172 - accuracy: 0.0926
Epoch 7/150
567/567 [==============================] - 7s 13ms/step - loss: 5.2435 - accuracy: 0.1066
Epoch 8/150
567/567 [==============================] - 7s 13ms/step - loss: 5.1131 - accuracy: 0.1134
Epoch 9/150
567/567 [==============================] - 7s 13ms/step - loss: 4.9990 - accuracy: 0.1170
Epoch 10/150
567/567 [==============================] - 7s 12ms/step - loss: 4.86

In [33]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model3.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [34]:
seed_text = 'Había una '
generate_poetry(seed_text, 50)

vez un señor que recibió el encargo de hacer una señora lleva a este hombre para que se almacena o
sea que el hombre con el que quería casarse debería ser guapo el hombre le dice la señora ya lo
que hacían esas cosas primero se rió y luego se enojó le puede usted y comenzó a arrojárselas para nadie
pero como ya estaba muy cansado va soltando lo que el hombre ya no sabe qué hacer está muy traje
un palo ¡mire le contesta el coyote ¿qué vamos a hacer con esta mesa esta vez se va caminando y
así lo hizo finalmente los dos perros obtuvieron alimento por ese palo el coyote le dice al coyote métete los
hombres porque no cumplió su ayuno pues el tlacuache no lo entiende así no podía agarrar a ninguno me quieres
el coyote baja la cabeza y dice si me quieres dar su hijo y no tardó el palo al suelo
se revolcaba del dolor mientras ya se escapó el peñasco sobre de ir a su grupo se sentó y la
gallina aleteó y corrió asustada pero al no ver por donde me racionas el trabajo y le dijo contestó nos
traj